In [1]:
import numpy as np
import pandas as pd
import xgboost as xgb
from xgboost.sklearn import XGBClassifier
from sklearn.cross_validation import train_test_split
from sklearn.model_selection import KFold
from sklearn.utils import shuffle

/home/centos/anaconda2/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
filepath=unicode('/mnt/nas/eeg/processed_data_singal_EEG/','utf8')
print filepath
import os
fileName = sorted(os.listdir(filepath))
len(fileName)

/mnt/nas/eeg/processed_data_Dream/


20

In [3]:
print fileName[18]

subject8.pkl


In [4]:
def encode1(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 1
        elif label[i] == "N3":
            label[i] = 1
        elif label[i] == "R":
            label[i] = 0
    return label
def encode2(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 0
    return label
def encode3(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 0
    return label
def encode4(label):
    for i in range(label.shape[0]):
        if label[i] == "W":
            label[i] = 0
        elif label[i] == "N1":
            label[i] = 1
        elif label[i] == "N2":
            label[i] = 2
        elif label[i] == "N3":
            label[i] = 3
        elif label[i] == "R":
            label[i] = 4
    return label
def balance_label(label_train,num_W,num_N1,num_N2,num_N3,num_R):
    w_W = (num_N1+num_N2+num_N3+num_R)*1.0 / (num_W+num_N1+num_N2+num_N3+num_R)
    w_N1 = 2 + (num_W+num_N2+num_N3+num_R)*1.0 / (num_W+num_N1+num_N2+num_N3+num_R)
    w_N2 = 2 + (num_W+num_N1+num_N3+num_R)*1.0 / (num_W+num_N1+num_N2+num_N3+num_R)
    w_N3 = 2 + (num_W+num_N1+num_N2+num_R)*1.0 / (num_W+num_N1+num_N2+num_N3+num_R)
    w_R = (num_W+num_N1+num_N2+num_N3)*1.0 / (num_W+num_N1+num_N2+num_N3+num_R)
    print w_W, w_N1,w_N2,w_N3,w_R
    weight = np.ones(label_train.shape[0])
    for i in range(label_train.shape[0]):
        if 0 == label_train[i]:
            weight[i] = w_W
        elif 1 == label_train[i]:
            weight[i] = w_N1
        elif 2 == label_train[i]:
            weight[i] = w_N2
        elif 3 == label_train[i]:
            weight[i] = w_N3
        elif 4 == label_train[i]:
            weight[i] = w_R
    return weight

In [5]:
n_splits = 5
kf = KFold(n_splits=n_splits, random_state=17, shuffle=True)
for train_index, test_index in kf.split(fileName):
    print("TRAIN:", train_index, "TEST:", test_index)

('TRAIN:', array([ 0,  1,  4,  5,  6,  7,  8,  9, 10, 11, 12, 15, 16, 17, 18, 19]), 'TEST:', array([ 2,  3, 13, 14]))
('TRAIN:', array([ 0,  2,  3,  4,  6,  7,  9, 10, 11, 13, 14, 15, 16, 17, 18, 19]), 'TEST:', array([ 1,  5,  8, 12]))
('TRAIN:', array([ 0,  1,  2,  3,  5,  6,  8,  9, 12, 13, 14, 15, 16, 17, 18, 19]), 'TEST:', array([ 4,  7, 10, 11]))
('TRAIN:', array([ 1,  2,  3,  4,  5,  6,  7,  8, 10, 11, 12, 13, 14, 15, 17, 19]), 'TEST:', array([ 0,  9, 16, 18]))
('TRAIN:', array([ 0,  1,  2,  3,  4,  5,  7,  8,  9, 10, 11, 12, 13, 14, 16, 18]), 'TEST:', array([ 6, 15, 17, 19]))


In [8]:
acc_final = 0
filepath1=unicode('/mnt/nas/eeg/processed_data/','utf8')
for train_index, test_index in kf.split(fileName):
    
    trainArr = np.array([])
    testArr = np.array([])
    for i in train_index:
        f = fileName[i]
#         print f
        data = pd.read_pickle(filepath+f)
#         label = pd.read_pickle(filepath1+f).label
#         data = data.join(label)
        train = data[data.label != '?']
        train = np.array(train)
        trainArr = np.append(trainArr, train)
    for i in test_index:
        f = fileName[i]
        #print f
        data = pd.read_pickle(filepath+f)
#         label = pd.read_pickle(filepath1+f).label
#         data = data.join(label)
        test = data[data.label != '?']
        test = np.array(test)
        testArr = np.append(testArr, test)
        
    trainArr = trainArr.reshape(-1, 3955)
    testArr = testArr.reshape(-1, 3955)
    label_train = trainArr[::,-1]
    label_test = testArr[::,-1]
#     print label_train, label_test, label_eva
    train = trainArr[::,:3954] 
#     temp = np.concatenate([train,train],axis=1)
#     temp = np.concatenate([temp,train],axis=1)
#     temp = np.concatenate([temp,train],axis=1)
#     temp[:-1,250:500] = train[:-1,::] - train[1:,::]
#     temp[1:,500:750] = train[1:,::] - train[:-1,::]
#     temp[2:,750:] = train[2:,::] - train[:-2,::]
#     train = temp
#     del temp
    
    test = testArr[::,:3954]
#     temp = np.concatenate([test,test],axis=1)
#     temp = np.concatenate([temp,test],axis=1)
# #     temp = np.concatenate([temp,test],axis=1)
#     temp[:-1,250:500] = test[:-1,::] - test[1:,::]
#     temp[1:,500:750] = test[1:,::] - test[:-1,::]
# #     temp[2:,750:] = test[2:,::] - test[:-2,::]
#     test = temp
#     del temp
    
    
    N_train = encode4(label_train)
    N_test = encode4(label_test)

    
#     train = train[np.where(N_train != 0)]
#     test = test[np.where(N_test != 0)]
#     label_train = label_train[np.where(N_train != 0)]
#     label_test = label_test[np.where(N_test != 0)]
    
#     X_train = train
#     X_eval = eva
#     Y_train = label_train
#     Y_eval = label_eva
    X_train, X_eval, Y_train, Y_eval = train_test_split(
                            train, label_train, test_size=0.1)
    print X_train.shape, X_eval.shape, Y_train.shape, Y_eval.shape
    
    num_W = np.where(0 == Y_train)[0].shape[0]
    num_N1 = np.where(1 == Y_train)[0].shape[0]
    num_N2 = np.where(2 == Y_train)[0].shape[0]
    num_N3 = np.where(3 == Y_train)[0].shape[0]
    num_R = np.where(4 == Y_train)[0].shape[0]
#     num_0 = np.where(0 == Y_train)[0].shape[0]
#     num_1 = np.where(1 == Y_train)[0].shape[0]
    
    
    w = balance_label(Y_train,num_W,num_N1,num_N2,num_N3,num_R)
#     w = balance_label(label_train,num_0,num_1)
    print Y_train.max(), Y_eval.max()
    xg_train = xgb.DMatrix(X_train, label = Y_train, weight = w)
    xg_eval = xgb.DMatrix(X_eval, label = Y_eval)
    xg_test = xgb.DMatrix(test)
    param = {
        'eta' : 0.008,
        'max_depth' : 3,
        'min_child_weight' : 1,
        'gamma' : 0.5,
        'subsample' : 0.3,
        'lambda' : 1,
        'colsample_bytree' : 0.3,
        'objective' : 'multi:softmax',
        'num_class' : 5,
#         'nthread' : -1,
        'silent' : False,
        'seed' : 1
    }
    watchlist = [(xg_train, 'train'),(xg_eval, 'test')]
    num_round = 10000
#     Y_train = encode2(Y_train)
#     Y_eval = encode2(Y_eval)
#     label_test = encode2(label_test)
    
    
    print "trianing the model"
    model = xgb.train(param, xg_train, num_round, watchlist, verbose_eval=300)#, early_stopping_rounds = 1000, verbose_eval=300)
    pred = model.predict(xg_test)

    del model
    acc = pred == label_test
    acc = acc.sum()*1.0 / acc.shape[0]
    print acc
    
    acc_final += acc
print acc_final / n_splits

(17456, 3954) (1940, 3954) (17456,) (1940,)
0.745531622365 2.81679651696 2.65725252062 2.89900320807 0.881416131989
4 4
trianing the model
[0]	train-merror:0.285954	test-merror:0.335567
[300]	train-merror:0.196821	test-merror:0.258763
[600]	train-merror:0.171557	test-merror:0.224742
[900]	train-merror:0.151419	test-merror:0.202062
[1200]	train-merror:0.136558	test-merror:0.186598
[1500]	train-merror:0.122485	test-merror:0.183505
[1800]	train-merror:0.109891	test-merror:0.176289
[2100]	train-merror:0.098688	test-merror:0.175258
[2400]	train-merror:0.088567	test-merror:0.174227
[2700]	train-merror:0.078665	test-merror:0.170619
[3000]	train-merror:0.068634	test-merror:0.169072
[3300]	train-merror:0.061018	test-merror:0.165464
[3600]	train-merror:0.054084	test-merror:0.164433
[3900]	train-merror:0.048258	test-merror:0.163918
[4200]	train-merror:0.04087	test-merror:0.164948
[4500]	train-merror:0.036057	test-merror:0.164948
[4800]	train-merror:0.030927	test-merror:0.163402
[5100]	train-merro

KeyboardInterrupt: 

In [8]:
prob = model.predict(xg_train, output_margin=True)
sec_train = np.zeros((Y_train.shape[0],6))
sec_train[:,:5] = prob
sec_train[0, 5] = 0
sec_train[1:,5:] = Y_train[:-1].reshape(-1,1)
sec_train = sec_train.reshape(-1,6)
sec_train = (sec_train - sec_train.mean()) / sec_train.var()

NameError: name 'model' is not defined

In [35]:
prob = model.predict(xg_eval, output_margin=True)
sec_eval = np.zeros((Y_eval.shape[0],6))
sec_eval[:,:5] = prob
sec_eval[0, 5] = 0
sec_eval[1:,5:] = Y_eval.reshape(-1,1)[:-1]
sec_eval = sec_eval.reshape(-1,6)
sec_eval = (sec_eval - sec_eval.mean()) / sec_eval.var()

In [36]:
prob = model.predict(xg_test, output_margin=True)
sec_test = np.zeros((label_test.shape[0],6))
sec_test[:,:5] = prob
sec_test[0, 5] = 0
sec_test[1:,5:] = pred.reshape(-1,1)[:-1]
sec_test = sec_test.reshape(-1,6)
sec_test = (sec_test - sec_test.mean()) / sec_test.var()

In [18]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [12]:
pred

array([0, 0, 0, ..., 0, 0, 0])

In [37]:
print sec_train.shape, sec_test.shape, sec_eval.shape
print sec_train[0], sec_test[0], sec_eval[0]

(17456, 6) (6040, 6) (1940, 6)
[-0.73462063  0.76162343 -0.24528588 -0.82430456  0.8256879  -0.28139034] [ 1.30500447 -0.12559804 -0.35945365 -0.50182343 -0.64764584 -0.31241185] [-0.28750188  0.93008021 -0.57893491 -0.82508172 -0.33887822 -0.28057286]


In [22]:
sec_train = xgb.DMatrix(sec_train, label = Y_train)
sec_eval = xgb.DMatrix(sec_eval, label = Y_eval)
sec_test = xgb.DMatrix(sec_test, label = label_test)

In [45]:
from sklearn.svm import SVC

sec_model = SVC(C = 2, kernel = 'linear')

sec_model.fit(sec_train.tolist(), Y_train.tolist())

pred = sec_model.predict(sec_eval)

#   del model
acc = pred == Y_eval.tolist()
acc = acc.sum()*1.0 / acc.shape[0]
print acc


pred = sec_model.predict(sec_test)

#   del model
acc = pred == label_test
acc = acc.sum()*1.0 / acc.shape[0]
print acc


0.824226804124
0.716390728477


In [9]:
print acc_final / 10

0.817069862808


In [ ]:
train[1:,-1] == label_train[:-1]

In [16]:
print acc_final / 5

0.799140003031


In [15]:
from sklearn.metrics import confusion_matrix
pred = pred.tolist()
label_test = label_test.tolist()
a = confusion_matrix(pred, label_test)

In [16]:
a

array([[ 67,   1,   0,   1,   2],
       [ 20,  28,  17,   0,  36],
       [  0,   6, 299, 136,   8],
       [  0,   0,   1, 218,   0],
       [ 13,  11,   9,   0, 143]])

In [23]:
a

array([[ 67,   1,   0,   1,   5],
       [ 12,  23,  21,   0,  14],
       [  0,   3, 293, 136,   2],
       [  0,   0,   1, 218,   0],
       [ 21,  19,  11,   0, 168]])

In [15]:
pred

[2.0, 2.0, 3.0, 3.0, 3.0, 3.0, 2.0]

In [19]:
testArr.shape

(7, 2829)

In [ ]:
10folds:
0.793273037567, data13, weighted by ratio
    param = {
        'eta' : 0.008,
        'max_depth' : 6,
        'min_child_weight' : 1,
        'gamma' : 2,
        'subsample' : 0.7,
        'lambda' : 0.3,
        'colsample_bytree' : 0.8,
        'objective' : 'multi:softmax',
        'num_class' : 5,
#         'nthread' : -1,
        'silent' : False,
        'seed' : 1
    }

In [ ]:
0.783428619109, data13
    param = {
        'eta' : 0.01,
        'max_depth' : 6,
        'min_child_weight' : 1,
        'gamma' : 2,
        'subsample' : 0.7,
        'lambda' : 0.3,
        'colsample_bytree' : 0.8,
        'objective' : 'multi:softmax',
        'num_class' : 5,
#         'nthread' : -1,
        'silent' : False,
        'seed' : 1
    }